# Getting started with Kubernetes and Elasticsearch
Read more about the API specifications:
- https://elasticsearch-py.readthedocs.io/
- https://github.com/kubernetes-client/python/blob/release-9.0/kubernetes/docs/

### Import python libraries

In [ ]:
from elasticsearch import Elasticsearch
from kubernetes import client, config
from ssl import create_default_context
import base64

### Create k8s client

In [ ]:
config.load_incluster_config()
corev1 = client.CoreV1Api()

### Retrieve the Elasticsearch password using the k8s client

In [ ]:
es_host='https://tigera-secure-es-http.tigera-elasticsearch.svc.cluster.local:9200'
kb_host='https://tigera-secure-kb-http.tigera-kibana.svc.cluster.local:5601/tigera-kibana'

user='elastic'
password_b64  = corev1.read_namespaced_secret(name='tigera-secure-es-elastic-user', namespace = 'tigera-elasticsearch').data[user]
password = base64.b64decode(password_b64).decode('utf-8')
print(password)
cert_b64  = corev1.read_namespaced_secret(name='tigera-secure-es-http-certs-public', namespace = 'tigera-elasticsearch').data["tls.crt"]
cert = base64.b64decode(cert_b64).decode('utf-8')
print(cert)
with open('elastic.crt', 'w') as f:
    f.write(cert)
    f.close()


### Instantiate the elasticsearch client

In [ ]:
context = create_default_context(cafile="elastic.crt")
es = Elasticsearch([es_host], 
                   http_auth=('elastic', password),
                  scheme='https',
                  port=9200,
                  ssl_context = context)


### Show Elasticsearch info
Show some global information about your cluster.


In [ ]:
es.info()

Gives more information about the health of the cluster:
- The status should be green.

- All shards should be assigned

- If the cluster is yellow, it is up and available, but not all data may be accessible or replicated.

In [ ]:
es.cluster.health()

In [ ]:
print(es.cat.health(v=True))

In [ ]:
print(es.cat.indices(v=True))

In [ ]:
print(es.cat.allocation(v=True))

In [ ]:
print(es.cat.nodes(v=True))

In [ ]:
print(es.cat.shards(v=True))

Check the cluster settings. This can be useful if you are debugging OIDC or certificate related stuff.

In [ ]:
es.cluster.get_settings()

In [ ]:
oidc_trace_log_settings = '{"transient" : {"logger.org.elasticsearch.xpack.security.authc.oidc": "trace"}}'
es.cluster.put_settings(oidc_trace_log_settings)

In [ ]:
es.